# Tutorial Pyspark

## Crear una sesión de Pyspark

In [ ]:
import findspark
findspark.init()  # Ayuda a encontrar Spark si no está en el PATH
from pyspark.sql import SparkSession
# Crear una sesión de Spark
spark = SparkSession.builder.appName("TutorialPySpark").getOrCreate()
# Verificar que Spark está funcionando
print(f"Versión de Spark: {spark.version}")

25/03/24 12:26:06 WARN Utils: Your hostname, MacBook-Pro-de-Alain.local resolves to a loopback address: 127.0.0.1; using 192.168.1.143 instead (on interface en0)
25/03/24 12:26:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 12:26:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Versión de Spark: 3.5.5


# Creamos un dataframe

In [3]:
# Crear un DataFrame con datos manuales
datos = [("Juan", 25, "México"), 
         ("Ana", 30, "España"), 
         ("Luis", 35, "Argentina")]
columnas = ["Nombre", "Edad", "País"]
df = spark.createDataFrame(datos, columnas)
# Mostrar el DataFrame
df.show()

+------+----+---------+
|Nombre|Edad|     País|
+------+----+---------+
|  Juan|  25|   México|
|   Ana|  30|   España|
|  Luis|  35|Argentina|
+------+----+---------+



## Leer un archivo .csv

In [4]:
df_csv = spark.read.csv("datos.csv", header=True, inferSchema=True)
df_csv.show()

+------+----+---------+
|Nombre|Edad|     País|
+------+----+---------+
|  Juan|  25|   México|
|   Ana|  30|   España|
|  Luis|  35|Argentina|
+------+----+---------+



## Operaciones báscias con Dataframes

In [5]:
# Ver esquema del DataFrame
df.printSchema()

# Seleccionar una sola columna
df.select("Nombre").show()

# Filtrar datos (Edad > 28)
df.filter(df["Edad"] > 28).show()

# Contar registros por grupo
df.groupBy("País").count().show()

root
 |-- Nombre: string (nullable = true)
 |-- Edad: long (nullable = true)
 |-- País: string (nullable = true)

+------+
|Nombre|
+------+
|  Juan|
|   Ana|
|  Luis|
+------+

+------+----+---------+
|Nombre|Edad|     País|
+------+----+---------+
|   Ana|  30|   España|
|  Luis|  35|Argentina|
+------+----+---------+

+---------+-----+
|     País|count|
+---------+-----+
|   México|    1|
|   España|    1|
|Argentina|    1|
+---------+-----+



## Agregar, modificar y transformar columnas

In [6]:
from pyspark.sql.functions import col, upper

# Agregar una nueva columna con la edad en 10 años
df = df.withColumn("Edad+10", df["Edad"] + 10)

# Convertir la columna "País" a mayúsculas
df = df.withColumn("País_MAYUS", upper(col("País")))

df.show()

+------+----+---------+-------+----------+
|Nombre|Edad|     País|Edad+10|País_MAYUS|
+------+----+---------+-------+----------+
|  Juan|  25|   México|     35|    MÉXICO|
|   Ana|  30|   España|     40|    ESPAÑA|
|  Luis|  35|Argentina|     45| ARGENTINA|
+------+----+---------+-------+----------+



## Uso de SQL en PySpark

In [7]:
# Registrar el DataFrame como tabla SQL
df.createOrReplaceTempView("personas")
# Ejecutar una consulta SQL
resultado = spark.sql("SELECT Nombre, Edad FROM personas WHERE Edad > 28")
resultado.show()


+------+----+
|Nombre|Edad|
+------+----+
|   Ana|  30|
|  Luis|  35|
+------+----+



## Guardar y exportar datos

In [8]:
# Guardar en formato CSV
df.write.csv("salida.csv", header=True)

# Guardar en formato Parquet (más eficiente)
df.write.parquet("salida.parquet")